<a href="https://colab.research.google.com/github/vifirsanova/compling/blob/main/nmt_data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

`Курс "Компьютерная лингвистика" | НИУ ВШЭ Санкт-Петербург 2024 (c) В.И. Фирсанова`

# Нейросетевой машинный перевод: подготовка данных к машинному обучению

План занятия:

I. Знакомимся с процессом подготовки данных к машинному обучению

II. Вместе проходим туториал TensorFlow и осваиваем модель кодер-декодер на примере рекуррентной нейросети

## Загрузка данных

In [1]:
!wget https://raw.githubusercontent.com/vifirsanova/compling/main/data/toy_data.en
!wget https://raw.githubusercontent.com/vifirsanova/compling/main/data/toy_data.ru

--2024-02-09 08:53:33--  https://raw.githubusercontent.com/vifirsanova/compling/main/data/toy_data.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1590 (1.6K) [text/plain]
Saving to: ‘toy_data.en’

toy_data.en         100%[===================>]   1.55K  --.-KB/s    in 0s      

2024-02-09 08:53:33 (21.3 MB/s) - ‘toy_data.en’ saved [1590/1590]

--2024-02-09 08:53:33--  https://raw.githubusercontent.com/vifirsanova/compling/main/data/toy_data.ru
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2501 (2.4K) [text/plain]
Saving to: ‘

Импорт библиотек

In [2]:
import numpy as np

from sklearn.model_selection import train_test_split

import tensorflow as tf

Запись данных в переменные

In [3]:
def load_data(path):
  with open(path, 'r', encoding='utf-8') as f:
    return np.array(f.read().split('\n'))

In [4]:
x_data, y_data = load_data('toy_data.ru'), load_data('toy_data.en')

print("Данные языка X:\n", x_data[5])
print("Данные языка Y:\n", y_data[5])

Данные языка X:
     Я люблю тебя.
Данные языка Y:
     I love you.


Создаем выборки

In [5]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data)

print("Тестовые данные языка X:\n", x_test)
print("Тестовые данные языка Y:\n", y_test)

Тестовые данные языка X:
 ['    Что ты хочешь сказать?' '    Какие у тебя планы на будущее?'
 '    Что ты хочешь сделать в будущем?' '    Что ты думаешь о этом месте?'
 '    Привет!' '    Какая твоя профессия?'
 '    Какие фильмы ты любишь смотреть?'
 '    Как ты относишься к этому предложению?'
 '    Что ты собираешься делать сегодня?' '    Я не понимаю.'
 '    Пожалуйста.' '    Какая твоя любимая песня?'
 '    Какой прекрасный день!']
Тестовые данные языка Y:
 ['    What do you want to say?' '    What are your plans for the future?'
 '    What do you want to do in the future?'
 '    What do you think about this place?' '    Hello!'
 '    What is your profession?' '    What movies do you like to watch?'
 '    What is your attitude toward this proposal?'
 '    What are you going to do today?' "    I don't understand."
 "    You're welcome." '    What is your favorite song?'
 '    What a beautiful day!']


## Кодирование

Создаем объекты Dataset

**Тензор** - объект векторного пространства V конечной размерности n

*shape* - размерность тензора

*dtype* - какие данные хранятся в объекте

**Батч** - некий отрезок данных; на батчи обычно деляться данные для машинного обучения

In [6]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(64).shuffle(len(x_data))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(64).shuffle(len(x_data))

for x_strings, y_strings in train_dataset.take(1):
  print("Образец tf-датасета, данные языка X:\n", x_strings[1:3])
  print("\nОбразец tf-датасета, данные языка Y:\n", y_strings[1:3])

Образец tf-датасета, данные языка X:
 tf.Tensor(
[b'    \xd0\xa7\xd1\x82\xd0\xbe \xd1\x82\xd1\x8b \xd1\x85\xd0\xbe\xd1\x82\xd0\xb5\xd0\xbb \xd0\xb1\xd1\x8b \xd0\xb7\xd0\xb0\xd0\xba\xd0\xb0\xd0\xb7\xd0\xb0\xd1\x82\xd1\x8c \xd0\xb2 \xd1\x8d\xd1\x82\xd0\xbe\xd0\xbc \xd1\x80\xd0\xb5\xd1\x81\xd1\x82\xd0\xbe\xd1\x80\xd0\xb0\xd0\xbd\xd0\xb5?'
 b'    \xd0\xa1\xd0\xbf\xd0\xb0\xd1\x81\xd0\xb8\xd0\xb1\xd0\xbe!'], shape=(2,), dtype=string)

Образец tf-датасета, данные языка Y:
 tf.Tensor([b'    What would you like to order in this restaurant?' b'    Thank you!'], shape=(2,), dtype=string)


Лирическое отступление: чем матрица отличается от тензора?

Тензор - широкое понятие, в зависимости от его размерности выделяют:

- скалярные тензоры (хранит ровно одно значение); `shape ()`

- вектора (одномерный список значений любой длины);  `shape (n, )`

- матрицы (двумерный массив, как таблица); `shape (n, m)`

- тензоры (любая глубина); например `shape (n, m, k)` будет выглядеть как параллелепипед или как 3 состыкованные вместе матрицы (таблицы)

In [25]:
print("Скалярный тензор с числом 42:\n", tf.constant(42))
print("\n Вектор длины n=2 с числами 42 и 24:\n", tf.constant([42, 24]))
print("\n Матрица длины содержит m=3 вектора с числами 42 и 24 длины n=2:\n", tf.constant([[42, 24],
                                                                                      [42, 24],
                                                                                      [42, 24]]))
print("\n Трёхмерный тензор содержит k=2 матрицы; \
матрица содержит m=3 вектора с числами 42 и 24 длины n=2:\n", tf.constant([[[42, 24], [42, 24], [42, 24]],
                                                                           [[42, 24], [42, 24], [42, 24]]]))
# NB! 32 бита - объем занимаемой памяти, типы данных можно менять

Скалярный тензор с числом 42:
 tf.Tensor(42, shape=(), dtype=int32)

 Вектор длины n=2 с числами 42 и 24:
 tf.Tensor([42 24], shape=(2,), dtype=int32)

 Матрица длины содержит m=3 вектора с числами 42 и 24 длины n=2:
 tf.Tensor(
[[42 24]
 [42 24]
 [42 24]], shape=(3, 2), dtype=int32)

 Трёхмерный тензор содержит k=2 матрицы; матрица содержит m=3 вектора с числами 42 и 24 длины n=2:
 tf.Tensor(
[[[42 24]
  [42 24]
  [42 24]]

 [[42 24]
  [42 24]
  [42 24]]], shape=(2, 3, 2), dtype=int32)


Теперь немного посчитаем:

In [28]:
a = tf.constant([[1, 2],
                 [3, 4]])

b = tf.constant([[1, 1],
                 [1, 2]])

print("Сложение:\n", tf.add(a, b), "\n")
print("Умножение (поэлементное):\n", tf.multiply(a, b), "\n")
print("Умножение матриц:\n", tf.matmul(a, b), "\n")

# NB! К тензорам можно применять и другие функции, например, argmax

Сложение:
 tf.Tensor(
[[2 3]
 [4 6]], shape=(2, 2), dtype=int32) 

Умножение (поэлементное):
 tf.Tensor(
[[1 2]
 [3 8]], shape=(2, 2), dtype=int32) 

Умножение матриц:
 tf.Tensor(
[[ 3  5]
 [ 7 11]], shape=(2, 2), dtype=int32) 



Обычно используют прямоугольные тензоры, т.е. количество элементов в каждом компоненте должно совпадать, но бывают исключения, например, `RaggedTensor`.

In [35]:
print('Без ragged мы получим ошибку!')

tf.ragged.constant([
                    ['this', 'is', 'a', 'sample', 'sentence'],
                    ['this', 'sentence', 'is', 'shorter']
                    ])

Без ragged мы получим ошибку!


<tf.RaggedTensor [[b'this', b'is', b'a', b'sample', b'sentence'],
 [b'this', b'sentence', b'is', b'shorter']]>

Тензоры бывают плотные `dense` и `разреженные`.

С разреженными векторами матрицами мы работаем, например, при использовании метода `one-hot encoding`.

In [59]:
print("Закодируем число 128 с помощью one-hot encoding\n")
sparse_tensor = tf.sparse.SparseTensor(indices=[[0, 1], [1, 2], [2, 8]],
                                       values=[1, 1, 1],
                                       dense_shape=[3, 10])

print("Метод sparse помогает нам сэкономить память компьютера", sparse_tensor, "\n")
print("Вот, как выглядит наша матрица на самом деле:\n", tf.sparse.to_dense(sparse_tensor))

Закодируем число 128 с помощью one-hot encoding

Метод sparse помогает нам сэкономить память компьютера SparseTensor(indices=tf.Tensor(
[[0 1]
 [1 2]
 [2 8]], shape=(3, 2), dtype=int64), values=tf.Tensor([1 1 1], shape=(3,), dtype=int32), dense_shape=tf.Tensor([ 3 10], shape=(2,), dtype=int64)) 

Вот, как выглядит наша матрица на самом деле:
 tf.Tensor(
[[0 1 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0]], shape=(3, 10), dtype=int32)


## Предобработка данных

Чистка и токенизация

In [7]:
def tokenize(text):
  # к нижнему регистру
  text_lower = tf.strings.lower(text, encoding='utf-8')
  # оставим знаки препинания и все буквы латиницы и кириллицы
  text_clean = tf.strings.regex_replace(text_lower, '[^ a-zа-я.?!,]', '')
  # добавим пробел перед знаками препинания (для токенизации знаков)
  text_punct = tf.strings.regex_replace(text_clean, '[.?!,]', r' \0 ')
  # избавимся от лишних пробелов
  text = tf.strings.strip(text_punct)
  # добавим метки начала (sos: start of the sentence) и конца (eos: end of the sentence) предложений
  return tf.strings.join(['<SOS>', text, '<EOS>'], separator=' ')

In [8]:
print("Образец токенизации:\n", tokenize(tf.constant("Как твои дела?")).numpy().decode())

Образец токенизации:
 <SOS> как твои дела ? <EOS>


Кодирование (векторизация): готовим данные для "скармливания" их модели

In [9]:
x_text_processor = tf.keras.layers.TextVectorization(standardize=tokenize, ragged=True)

x_text_processor.adapt(train_dataset.map(lambda x, target: x))
print("Образец обучающего словаря языка X:\n", x_text_processor.get_vocabulary()[:10])
print("\nОбразец закодированного текста X:\n\tТекст:", x_strings[1].numpy().decode(), "\n\tКодирование:", x_text_processor(x_strings[1]))

y_text_processor = tf.keras.layers.TextVectorization(standardize=tokenize, ragged=True)

y_text_processor.adapt(train_dataset.map(lambda x, target: target))
print("\nОбразец обучающего словаря языка Y:\n", y_text_processor.get_vocabulary()[:10])
print("\nОбразец закодированного текста X:\n\tТекст:", y_strings[1].numpy().decode(), "\n\tКодирование:", y_text_processor(y_strings[1]))

Образец обучающего словаря языка X:
 ['', '[UNK]', '<SOS>', '<EOS>', '?', 'ты', 'что', 'как', 'тебя', 'у']

Образец закодированного текста X:
	Текст:     Что ты хотел бы заказать в этом ресторане? 
	Кодирование: tf.Tensor([ 2  6  5 40 98 75 11 34 52  4  3], shape=(11,), dtype=int64)

Образец обучающего словаря языка Y:
 ['', '[UNK]', '<SOS>', '<EOS>', '?', 'you', 'what', 'do', 'how', 'your']

Образец закодированного текста X:
	Текст:     What would you like to order in this restaurant? 
	Кодирование: tf.Tensor([ 2  6 34  5 24 11 55 14 21 51  4  3], shape=(12,), dtype=int64)


In [10]:
def encode(x, y):
  # обработка текстов (текст -> словарь -> замена слов их ID'шками из словаря)
  x_encoded = x_text_processor(x).to_tensor()
  y_encoded = y_text_processor(y)
  # (x_encoded, y_encoded) -> ((x_encoded, y_input), y_label) для keras.Model.fit
  # Keras принимает (inputs, labels)
  # inputs = (x_encoded, y_input)
  # labels = y_label
  # y_label += следующий за y_input токен
  y_input = y_encoded[:,:-1].to_tensor()
  y_label = y_encoded[:,1:].to_tensor()
  return (x_encoded, y_input), y_label

In [11]:
train = train_dataset.map(encode, tf.data.AUTOTUNE)
test = test_dataset.map(encode, tf.data.AUTOTUNE)

for (x_encoded_token, y_input_token), y_label_token in train.take(1):
  print("Закодированный текст x:")
  print(x_encoded_token[1, :10].numpy())
  print("\nЗакодированный текст y:")
  print(y_input_token[1, :10].numpy())
  print("\nТот же текст y мы сдвинули на 1 токен вперед:")
  print(y_label_token[1, :10].numpy())

Закодированный текст x:
[ 2  6  5 40 98 75 11 34 52  4]

Закодированный текст y:
[ 2  6 34  5 24 11 55 14 21 51]

Тот же текст y мы сдвинули на 1 токен вперед:
[ 6 34  5 24 11 55 14 21 51  4]


## Домашнее задание

Задание:

1. **По желанию**

* Ознакомиться с [туториалом TensorFlow](https://www.tensorflow.org/text/tutorials/nmt_with_attention) по машинному переводу

* Адаптировать препроцессинг на основе этого воркбука и обучить модель из туториала TensorFlow на данных `toy_data.ru` и `toy_data.en`

2. **Обязательно**

* Ниже: образец подсчета BLEU-Score для системы машинного перевода. Вспомните, как считается BLEU и опишите своими словами форумулу расчета и интерпретацию результатов. Ответьте на следующие вопросы:

  * Что значит кандидат и референс?
  * Для чего подойдет BLEU-Score: подсчета отдельных переводов или результатов работы системы на целом корпусе?
  * Какие "скоры" считаются высокими, а какие - низкими?
  
* Проанализируйте результаты в ячейках ниже. Порассуждайте:

  * На ваш взгляд, какие слова, контексты вызвали трудности у автоматической системы?
  * Руководствуясь знаниями о том, как работают нейросети, предположите, что вызвало ошибки.

In [ ]:
!pip install evaluate

In [13]:
import evaluate

predictions = [
    "In addition to the usual hockey, there is an underwater and even ice version of this game.", #  Помимо обычного хоккея, существует подводная и даже подлёдная разновидности этой игры.
    "You reap what you sow.", # Что посеешь - то и пожнешь.
    "Also, justice is how we distribute the small resources." # Также, справедливость - это и то, как мы распределяем малые ресурсы.
    ]

references = [
    ["In addition to conventional hockey, there is an underwater and even under-ice variety of this game."], #  Помимо обычного хоккея, существует подводная и даже подлёдная разновидности этой игры.
    ["What goes around comes around."], # Что посеешь - то и пожнешь.
    ["Fairness also has to do with how we distribute scarce resources."], # Также, справедливость - это и то, как мы распределяем малые ресурсы.
    ]

bleu = evaluate.load("bleu")
results = bleu.compute(predictions=predictions, references=references)

print(results)

{'bleu': 0.374360558192794, 'precisions': [0.5833333333333334, 0.45454545454545453, 0.3333333333333333, 0.2222222222222222], 'brevity_penalty': 1.0, 'length_ratio': 1.0, 'translation_length': 36, 'reference_length': 36}
